In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [11]:
import os
os.chdir("D:/Projects/insect-id")

import torch
from executorch.exir import to_edge_transform_and_lower
from executorch.backends.xnnpack.partition.xnnpack_partitioner import XnnpackPartitioner

# import mynnlib
# from mynnlib import *

In [12]:
import torchvision
print(torchvision.__file__)

AttributeError: partially initialized module 'torchvision' has no attribute 'extension' (most likely due to a circular import)

# Generate .pt and classes json

In [6]:
output_dir = "models"
models = {
    "moth": "insect-dataset/moth/checkpoint.moth.tg.ep060001.pth",
    "butterfly": "insect-dataset/butterfly/checkpoint.butterfly.te.ep050000.pth",
    "lepidoptera": "insect-dataset/lepidoptera/checkpoint.lepidoptera.te.ep060004.pth",
    "lepidoptera.v2alpha": "insect-dataset/lepidoptera/checkpoint.inc.lepidoptera.ta.i01.e19.pth",
    "odonata": "insect-dataset/odonata/checkpoint.odonata.tc.ep060002.pth",
    "cicada": "insect-dataset/cicada/checkpoint.cicada.te.ep060000.pth",
    "root-classifier": "insect-dataset/root-classifier/checkpoint.root-classifier.tj.ep060001.pth"
}

In [10]:
for model_name, model_path in models.items():
    if os.path.exists(model_path):
        print(f"{model_path}")
        model_data = torch.load(model_path, weights_only=False)
        model = model_data['model']
        device = torch.device("cpu")
        model.to(device)
        model.eval()
        
        # ----- for excutorch android (requires python <= 3.12)
        example_input = torch.randn(1, 3, 224, 224)
        exported_program = torch.export.export(model, (example_input,))
        executorch_program = to_edge_transform_and_lower(exported_program, partitioner=[XnnpackPartitioner()]).to_executorch()
        pte_model_path = f"{output_dir}/m.checkpoint.{model_name}.pte"
        with open(pte_model_path, "wb") as f:
            f.write(executorch_program.buffer)
        print(f" --> {pte_model_path}")

        # ----- for pytorch android
        # scripted_model = torch.jit.script(model)
        # scripted_model_path = f"{output_dir}/m.checkpoint.{model_name}.pt"
        # scripted_model.save(scripted_model_path)
        # print(f" --> {scripted_model_path}")

        class_file_path = f"{output_dir}/classes.{model_name}.json"
        with open(class_file_path, "w", encoding="utf-8") as file:
            json.dump(model_data['class_names'], file, indent=4)
        print(f" --> {class_file_path}")

metadata = load_json("insect-id-app/metadata.json")
metadata['root-classifier']['classes'] = load_json("models/classes.root-classifier.json")
dump_json("insect-id-app/metadata.json", metadata)
os.remove("models/classes.root-classifier.json")

insect-dataset/moth/checkpoint.moth.tg.ep060001.pth


AttributeError: partially initialized module 'torchvision' has no attribute 'extension' (most likely due to a circular import)

# Model stats

In [5]:
base_dir = "insect-dataset"
metadata_path = "insect-id-app/metadata.json"

In [11]:
metadata = load_json(metadata_path)
for species_type in ['lepidoptera', 'moth', 'butterfly', 'odonata', 'cicada']:
    data_dir = f"insect-dataset/{species_type}/data"
    if os.path.exists(data_dir):
        if not metadata[species_type]['stats']:
            metadata[species_type]['stats'] = {}
        stats = metadata[species_type]['stats']
        stats['class_count'] = len(os.listdir(f"{data_dir}"))
        stats['species_count'] = len([class_name for class_name in os.listdir(f"{data_dir}") if not re.match(r"^.*-(early|genera|spp)$", class_name)])
        stats['spp_class_count'] = len([class_name for class_name in os.listdir(f"{data_dir}") if re.match(r"^.*-(genera|spp)$", class_name)])
        stats['early_stage_class_count'] = len([class_name for class_name in os.listdir(f"{data_dir}") if re.match(r"^.*-(early)$", class_name)])
        stats['data_count'] = sum([len(os.listdir(f"{data_dir}/{class_name}")) for class_name in os.listdir(f"{data_dir}")])
dump_json(metadata_path, metadata)

In [ ]:
# model size calculation

metadata = load_json(metadata_path)
for model_name in ['root-classifier', 'lepidoptera.v2alpha', 'lepidoptera', 'moth', 'butterfly', 'odonata', 'cicada']:
    metadata[model_name]['size'] = 0
    for file in [f"{output_dir}/m.checkpoint.{model_name}.pt", f"{output_dir}/images.{model_name}.zip", 
                    f"{output_dir}/classes.{model_name}.json", f"{output_dir}/class_details.{model_name}.json"]:
        if os.path.exists(file):
            metadata[model_name]['size'] += os.path.getsize(file)
        else:
            # for lepidoptera.v2alpha
            file = file.replace(model_name, model_name.split('.')[0])
            if os.path.exists(file):
                metadata[model_name]['size'] += os.path.getsize(file)
dump_json(metadata_path, metadata)

In [13]:
def dump_stats(species_type):
    min_val_data_cnt = 2
    metadata = load_json(metadata_path)
    print(f"\n{species_type}\n" + ('-' * 30))

    # make val dataset uniform
    dataset_dir = f"{base_dir}/{species_type}"
    for class_name in os.listdir(f"{dataset_dir}/data"):
        if not os.path.exists(f"{dataset_dir}/val/{class_name}"):
            os.makedirs(f"{dataset_dir}/val/{class_name}")
        val_data_cnt = len(os.listdir(f"{dataset_dir}/val/{class_name}"))
        data_to_add = max(0, min_val_data_cnt - val_data_cnt)
        if data_to_add > 0:
            files = os.listdir(f"{dataset_dir}/data/{class_name}")
            random.shuffle(files)
            for file in files[:data_to_add]:
                shutil.copy2(f"{dataset_dir}/data/{class_name}/{file}", f"{dataset_dir}/val/{class_name}/{file}")

    # get accuracy
    model_data = torch.load(models[species_type], weights_only=False)
    top1 = validate_prediction_in_dir_top_k(f"{base_dir}/{species_type}/val", model_data, 1)
    print(f"Top 1 Success: {top1['success']}/{top1['total']} -> {100*top1['success']/top1['total']:.2f}%")
    top3 = validate_prediction_in_dir_top_k(f"{base_dir}/{species_type}/val", model_data, 3)
    print(f"Top 3 Success: {top3['success']}/{top3['total']} -> {100*top3['success']/top3['total']:.2f}%")

    # save in json
    if not metadata[species_type]['stats']:
        metadata[species_type]['stats'] = {}
    stats = metadata[species_type]['stats']
    stats['accuracy'] = f"{100*top1['success']/top1['total']:.2f}%"
    stats['accuracy_top3'] = f"{100*top3['success']/top3['total']:.2f}%"
    dump_json(metadata_path, metadata)

In [15]:
dump_stats("lepidoptera")


lepidoptera
------------------------------
Top 1 Success: 10667/11107 -> 96.04%
Top 3 Success: 11038/11107 -> 99.38%


In [16]:
dump_stats("moth")


moth
------------------------------
Top 1 Success: 5692/6541 -> 87.02%
Top 3 Success: 6204/6541 -> 94.85%


In [17]:
dump_stats("butterfly")


butterfly
------------------------------
Top 1 Success: 3017/3467 -> 87.02%
Top 3 Success: 3302/3467 -> 95.24%


In [18]:
dump_stats("odonata")


odonata
------------------------------
Top 1 Success: 1416/1701 -> 83.25%
Top 3 Success: 1589/1701 -> 93.42%


In [19]:
dump_stats("cicada")


cicada
------------------------------
Top 1 Success: 280/449 -> 62.36%
Top 3 Success: 317/449 -> 70.60%
